In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import re

In [7]:
base_url = '''https://www.camara.leg.br/busca-portal?contextoBusca=BuscaProposicoes&pagina={}
                                                    &order=data
                                                    &abaEspecifica=true
                                                    &filtros=%5B%7B"ano"%3A2020%7D%5D
                                                    &tipos=PL'''

In [8]:
driver = webdriver.Chrome('/home/victor/Documents/chromedriver')
driver.get(base_url.format(1))

In [10]:
#pl_urls = []
for i in range(67, 69):
    print(i)
    driver.get(base_url.format(i))
    links = driver.find_elements_by_css_selector('a')
    links = [e.get_attribute('href') for e in links]
    links = [e for e in links if e != None]
    links = [e for e in links if '/proposicoesWeb/' in e]
    
    pl_urls.extend(links)
pl_urls = list(set(pl_urls))


67
68


In [19]:
 class PL:
    colnames = [
        'nome',
        'inteiro_teor_url',
        'situacao',
        'autor_nome',
        'autor_partido',
        'autor_sigla_uf',
        'data_apresentacao',
        'ementa',
        'url'
    ]
    
    
    driver = ''
    nome = ''
    inteiro_teor_url = ''
    situacao = ''
    autor = {}
    data_apresentacao = ''
    ementa = ''
    url = ''
    
    def __init__(self, url):
        self.setURL(url)
        self.startDriver()
        self.setNome()
        self.setInteiroTeorUrl()
        self.setSituacao()
        self.setAutor()
        self.setDataApresentacao()
        self.setEmenta()
    def __init__(self):
        pass
    
    def setURL(self, url):
        self.url = url
        
        
    def startDriver(self):
        self.driver = driver
        driver.get(self.url)
    
    
    def setNome(self):
        self.nome = driver.find_element_by_css_selector('span.nomeProposicao').text
        
        
    def setInteiroTeorUrl(self):
        inteiro_teor = driver.find_element_by_css_selector('#content > h3.inteiroTeor > span.naoVisivelNaImpressao > a')
        self.inteiro_teor_url = inteiro_teor.get_attribute('href')
        
        
    def setSituacao(self):
        situacao = driver.find_element_by_css_selector('#subSecaoSituacaoOrigemAcessoria > p').text
        self.situacao = situacao.replace('Situação: ', '')
        
        
    def setAutor(self):
        autor = driver.find_element_by_css_selector('#colunaPrimeiroAutor').text
        autor = autor.replace('Autor\n', '')
        if not '/' in autor:
            self.autor = {
                'nome':autor,
                'partido':autor,
                'sigla_uf':autor
            }
        else:
            autor = autor.split(' - ')
            self.autor = {
                'nome':' - '.join(autor[:-1]),
                'partido':''.join(autor[-1]).split('/')[0],
                'sigla_uf':''.join(autor[-1]).split('/')[1]
            }

        
    def setDataApresentacao(self):
        data = driver.find_element_by_css_selector('#identificacaoProposicao > div:nth-child(1) > div > div.col-md-3 > p').text
        self.data_apresentacao = re.search('\d{2}/\d{2}/\d{4}', data).group(0)
    
    
    def setEmenta(self):
        self.ementa = driver.find_element_by_css_selector('#identificacaoProposicao > p > span').text
        
        
    def getAsTuple(self):
        return (self.nome,
        self.inteiro_teor_url,
        self.situacao,
        self.autor['nome'],
        self.autor['partido'],
        self.autor['sigla_uf'],
        self.data_apresentacao,
        self.ementa,
        self.url)

In [13]:
#pl_objects = []
fail = []
for (i, url) in enumerate(a):
    print(i)
    try:
        pl_objects.append(PL(url))
    except:
        print('fail')
        fail.append(url)
        
a = list(set(fail) - set(pl_objects))
fail = 0

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [22]:
data = pl_objects
data = [e.getAsTuple() for e in pl_objects]
data = pd.DataFrame(data, columns=PL().colnames)
#data.to_csv('../data/pl_df.csv', index=False)

In [12]:
pl_df = pd.read_csv('../data/pl_df.csv')
pl_df

,nome,inteiro_teor_url,situacao,autor_nome,autor_partido,autor_sigla_uf,data_apresentacao,ementa,url
0,PL 1310/2020,https://www.camara.leg.br/proposicoesWeb/prop_...,Aguardando Despacho do Presidente da Câmara do...,Pompeo de Mattos,PDT,RS,31/03/2020,Dispõe sobre a concessão do auxílio-doença de ...,https://www.camara.leg.br/proposicoesWeb/ficha...
1,PL 733/2020,https://www.camara.leg.br/proposicoesWeb/prop_...,Aguardando Despacho do Presidente da Câmara do...,Eduardo Bismarck,PDT,CE,18/03/2020,Proíbe que as distribuidoras de energia interr...,https://www.camara.leg.br/proposicoesWeb/ficha...
2,PL 1393/2020,https://www.camara.leg.br/proposicoesWeb/prop_...,Aguardando Despacho do Presidente da Câmara do...,Evair Vieira de Melo,PP,ES,01/04/2020,Estabelece medidas excepcionais de proteção so...,https://www.camara.leg.br/proposicoesWeb/ficha...
3,PL 431/2020,https://www.camara.leg.br/proposicoesWeb/prop_...,Situação:,Alexandre Frota,PSDB,SP,03/03/2020,Cria a obrigatoriedade de repasses automáticos...,https://www.camara.leg.br/proposicoesWeb/ficha...
4,PL 765/2020,https://www.camara.leg.br/proposicoesWeb/prop_...,Aguardando Despacho do Presidente da Câmara do...,Reginaldo Lopes,PT,MG,19/03/2020,Autoriza o Poder Executivo a conceder perdão d...,https://www.camara.leg.br/proposicoesWeb/ficha...
5,PL 1324/2020,https://www.camara.leg.br/proposicoesWeb/prop_...,Aguardando Despacho do Presidente da Câmara do...,José Nelto,PODE,GO,31/03/2020,Dispõe sobre dispensação de prescrição de rece...,https://www.camara.leg.br/proposicoesWeb/ficha...
6,PL 1606/2020,https://www.camara.leg.br/proposicoesWeb/prop_...,Situação:,Julio Cesar Ribeiro,REPUBLIC,DF,06/04/2020,Dispõe sobre a fixação do preço do gás de cozi...,https://www.camara.leg.br/proposicoesWeb/ficha...
7,PL 678/2020,https://www.camara.leg.br/proposicoesWeb/prop_...,Aguardando Despacho do Presidente da Câmara do...,Felipe Carreras,PSB,PE,17/03/2020,Estabelece regras de prevenção da transmissão ...,https://www.camara.leg.br/proposicoesWeb/ficha...
8,PL 852/2020,https://www.camara.leg.br/proposicoesWeb/prop_...,Aguardando Despacho do Presidente da Câmara do...,Kim Kataguiri,DEM,SP,23/03/2020,Institui a suspensão do pagamento de jetons em...,https://www.camara.leg.br/proposicoesWeb/ficha...
9,PL 1212/2020,https://www.camara.leg.br/proposicoesWeb/prop_...,Aguardando Despacho do Presidente da Câmara do...,Ted Conti,PSB,ES,30/03/2020,Dispõe sobre a utilização extraordinária do Fu...,https://www.camara.leg.br/proposicoesWeb/ficha...


In [27]:
## filter
#keyword = query[0]
keyword
bool_array.extend([keyword in pl_df.loc[i, 'ementa'].lower() for i in pl_df.index])

[False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 Tru

In [82]:
def myFilter(query, pl_df):
    query = re.findall('"(.+?)"', query)

    bool_arrays = []
    for keyword in query:
        print(keyword)
        bool_arrays.append([keyword in pl_df.loc[i, 'ementa'].lower() for i in pl_df.index])
    bool_arrays = [any([bool_arrays[j][i] for j in range(len(bool_arrays))]) for i in range(len(bool_arrays[0]))]
    return bool_arrays

In [88]:
query = '"covid" "pandemia" "isolamento social" "quarentena" "corona" "calamidade"'
bool_array = myFilter(query, pl_df)

covid
pandemia
isolamento social
quarentena
corona
calamidade


In [89]:
pl_df = pl_df.loc[bool_array, :]

In [85]:
for i in pl_df.index:
    print(pl_df.loc[i, 'ementa'])
    print('___')

Estabelece medidas excepcionais de proteção social aos trabalhadores autônomos que enumera a serem adotadas durante o período de enfrentamento da emergência de saúde pública de importância inter-nacional decorrente do coronavírus respon-sável pelo surto de 2019
___
Autoriza o Poder Executivo a conceder perdão de dívida de hospitais privados sem fins lucrativos sem certificado de CEBAS que disponibilizarem leitos SUS para o tratamento da COVID-19.
___
Dispõe sobre dispensação de prescrição de receita para aquisição de medicamentos de uso contínuo em casos de calamidade pública e quarentena nos moldes da Lei 13.979 de 06 de fevereiro de 2020.
___
Institui a suspensão do pagamento de jetons em casos de pandemia e estado de calamidade pública.
___
Dispõe sobre a utilização extraordinária do Fundo Especial de Financiamento de Campanha, previsto na Lei n° 9.504, de 30 de setembro de 1997, no combate à pandemia de COVID-19.
___
Dispõe sobre regras aplicáveis ao trabalho doméstico em razão das

In [91]:
pl_df.to_excel('../data/Projetos_de_Lei_2020_COVID.xlsx', index=False)